In [57]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
from __future__ import absolute_import, division, print_function
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.regularizers import l2
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error
from keras.layers.advanced_activations import SReLU
print('The scikit-learn version is {}.'.format(sklearn.__version__))

The scikit-learn version is 0.18.


In [58]:
#Load the training and test files
df_train = pd.read_csv('../input/train.csv')
df_test = pd.read_csv('../input/test.csv')
print('training: ', df_train.shape)
print('test: ', df_test.shape)

training:  (188318, 132)
test:  (125546, 131)


In [59]:
cols = df_train.columns
cols_test = df_test.columns 

In [60]:
#Convert to Numpy arrays and separate features/targets
# X_train = df_train.as_matrix()
X_train = df_train
y_train = X_train[cols[-1]]
X_train = X_train[cols[1:-1]]

# X_test = df_test.as_matrix()
X_test = df_test
X_test = X_test[cols_test[1:]]

cols = X_train.columns
cols_test = X_test.columns 


In [61]:
#Variable to hold the list of labels for all category features in the train and test data
labels = []

for i in range(0,116):
    train = X_train[cols[i]].unique()
    test = X_test[cols[i]].unique()
    labels.append(list(set(train) | set(test)))    


In [62]:
#Import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

#One hot encode all categorical features - training set
cats = []
for i in range(0, 116):
    #Label encode
    label_encoder = LabelEncoder()
    label_encoder.fit(labels[i])
    feature = label_encoder.transform(X_train.iloc[:,i])
    feature = feature.reshape(X_train.shape[0], 1)
    #One hot encode
    onehot_encoder = OneHotEncoder(sparse=False,n_values=len(labels[i]))
    feature = onehot_encoder.fit_transform(feature)
    cats.append(feature)

# Make a 2D array from a list of 1D arrays
encoded_cats = np.column_stack(cats)

# Print the shape of the encoded data
print(encoded_cats.shape)

#Concatenate encoded attributes with continuous attributes
X_train = np.concatenate((encoded_cats,X_train.iloc[:,116:].values),axis=1)
del cats
del feature

del encoded_cats
print(X_train.shape)

(188318, 1176)
(188318, 1190)


In [63]:
#One hot encode all categorical features - test set
cats = []
for i in range(0, 116):
    #Label encode
    label_encoder = LabelEncoder()
    label_encoder.fit(labels[i])
    feature = label_encoder.transform(X_test.iloc[:,i])
    feature = feature.reshape(X_test.shape[0], 1)
    #One hot encode
    onehot_encoder = OneHotEncoder(sparse=False,n_values=len(labels[i]))
    feature = onehot_encoder.fit_transform(feature)
    cats.append(feature)

# Make a 2D array from a list of 1D arrays
encoded_cats = np.column_stack(cats)

# Print the shape of the encoded data
print(encoded_cats.shape)

#Concatenate encoded attributes with continuous attributes
X_test = np.concatenate((encoded_cats,X_test.iloc[:,116:].values),axis=1)
del cats
del feature

del encoded_cats
print(X_test.shape)

(125546, 1176)
(125546, 1190)


In [64]:
r, c = X_train.shape

#Split the data into chunks
from sklearn import cross_validation
X_train, X_val, y_train, y_val = cross_validation.train_test_split(X_train, y_train, test_size=0.2, random_state=0)

In [65]:
X_val.shape

(37664, 1190)

In [66]:
#from keras.layers.normalization import BatchNormalization

def larger_model():
    model = Sequential()
    model.add(Dense(400, input_dim = c, W_regularizer=l2(0.1), init = 'he_normal'))
    model.add(SReLU())
    model.add(Dropout(0.4))
    model.add(Dense(200, W_regularizer=l2(0.1), init = 'he_normal'))
    model.add(SReLU())
    model.add(Dropout(0.2))
    model.add(Dense(1, init = 'he_normal'))
    model.compile(loss = 'mae', optimizer = 'adadelta')

    return model

In [67]:
np.random.seed(0)


estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(build_fn=larger_model, nb_epoch=35, batch_size=40, verbose=1)))
pipeline = Pipeline(estimators)


pipeline.fit(X_train, y_train)


Epoch 1/35
150654/150654 [==============================] - 35s - loss: 1324.8700    
Epoch 2/35
150654/150654 [==============================] - 34s - loss: 1241.9735    
Epoch 3/35
150654/150654 [==============================] - 34s - loss: 1226.2003    
Epoch 4/35
150654/150654 [==============================] - 34s - loss: 1214.7448    
Epoch 5/35
150654/150654 [==============================] - 34s - loss: 1206.0821    
Epoch 6/35
150654/150654 [==============================] - 34s - loss: 1199.8560    
Epoch 7/35
150654/150654 [==============================] - 34s - loss: 1192.7763    
Epoch 8/35
150654/150654 [==============================] - 34s - loss: 1188.0551    
Epoch 9/35
150654/150654 [==============================] - 34s - loss: 1184.1263    
Epoch 10/35
150654/150654 [==============================] - 34s - loss: 1180.5522    
Epoch 11/35
150654/150654 [==============================] - 34s - loss: 1177.3071    
Epoch 12/35
150654/150654 [=========================

Pipeline(steps=[('standardize', StandardScaler(copy=True, with_mean=True, with_std=True)), ('mlp', <keras.wrappers.scikit_learn.KerasRegressor object at 0x7f63fb62c6a0>)])

In [68]:
pred_val = pipeline.predict(X_val)

result = mean_absolute_error(y_val, pred_val)
result                      
# Activation ..> val score

# SRelU(10) -> 1173.5
# SReLU(20) -> 1166.9
# SReLU(23) -> 1167.8
# SReLU(25) -> 1163.9 --> 1143
# SRelu(30) -> 1166.4 ->> 1146
# SReLU(60) -> 1173.0 --> 1155 (on kaggle)

# Reg(0.01), Epoch(25) -> 1161.7
# Reg(0.01), Epoch(30) -> 1164.6
# Reg(0.10), Epoch(30) -> 1158.9 --> 1143
# Reg(0.10), Epoch(35) 1118 -> 1159.6



37640/37664 [============================>.] - ETA: 0s

1159.6181284920847

In [69]:
pred_targets = pipeline.predict(X_test)

125440/125546 [============================>.] - ETA: 0s

In [70]:
# pred_targets

In [71]:

output = pd.DataFrame({
        "ID": df_test["id"],
        "loss": pred_targets[:]
    })
output.to_csv("../input/output.csv", index=False)

